In [154]:
import glob
import re
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report
import csv
from bs4 import BeautifulSoup
from nltk.corpus import stopwords
import nltk
nltk.download("stopwords")
stemmer = nltk.stem.SnowballStemmer('english')
# stemmer = nltk.stem.lancaster.LancasterStemmer()

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/mhilmiasyrofi/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [155]:
def clean_html(raw_html):
    cleantext = BeautifulSoup(raw_html, "lxml").get_text() 
    return cleantext

def preprocess(corpus):
    letters_only = re.sub("[^a-zA-Z]", " ", corpus) 
    
    # Convert to lower case, split into individual words
    words = letters_only.lower().split()
    
    stops = set(stopwords.words("english"))
    
    # Remove stop words
    meaningful_words = [w for w in words if not w in stops]   
    
    # stem words
    words = ([stemmer.stem(w) for w in words])

    # Join the words back into one string separated by space, 
    # and return the result.
    return( " ".join( meaningful_words )) 

In [156]:
with open("spam-mail.tr.label", "r") as file:
    lines = file.readlines()
    
lines = [l.strip() for l in lines[1:]]
labels = {}
for line in lines:
    x = int(line.split(',')[0])
    y = int(line.split(',')[1])
    labels[x] = y

In [ ]:
email_folder = "tr"
emails = glob.glob(email_folder + "/*")

train_x = []
train_y = []
test_x = []
test_y = []

for email in emails[:int(0.8* len(emails))]:
    with open(email, "r", encoding='iso-8859-1') as handle:
        email_html = handle.read()
        email_clean = clean_html(email_html)
        email_proprocess = preprocess(email_clean)
#         print(email_proprocess)
    
    train_x.append(email_html)
    id = int(email.split('.')[0].split('_')[1])
    train_y.append(labels[id])
    

for email in emails[int(0.8* len(emails)):]:
    with open(email, "r", encoding='iso-8859-1') as handle:
        email_html = handle.read()
        email_clean = clean_html(email_html)
        email_proprocess = preprocess(email_clean)
    
    test_x.append(email_html)
    id = int(email.split('.')[0].split('_')[1])
    test_y.append(labels[id])

In [ ]:
count_vect = CountVectorizer()
train_x_count = count_vect.fit_transform(train_x)

transformer = TfidfTransformer(smooth_idf=False)
train_x_count = transformer.fit_transform(train_x_count)

In [ ]:
clf = DecisionTreeClassifier().fit(train_x_count, train_y)

In [ ]:
test_x_count = count_vect.transform(test_x)
test_x_count = transformer.transform(test_x_count)
predicted = clf.predict(test_x_count)

count_true = 0
count_false = 0
for pred, label in zip(predicted, test_y):
    if pred == label:
        count_true += 1
    else:
        count_false += 1

In [ ]:
print(classification_report(y_true= test_y, y_pred= predicted, digits=4))